In [1]:
import pandas as pd
import plotly.express as px

# 🏠_Home

In [2]:
# Load Data

events_products = pd.read_csv('data/combined_events_products.csv')

In [3]:
events_products['date'] = pd.to_datetime(events_products['event_time']).dt.date

In [4]:
events_products.head()

,user_id,visit_id,event_name,sequence_number,event_time,page_name,product_category,date
0,1,ccf365,Page View,1,2020-02-04 19:16:09.182546,Home Page,NaN,2020-02-04
1,1,ccf365,Page View,2,2020-02-04 19:16:17.358191,All Products,NaN,2020-02-04
2,1,ccf365,Page View,3,2020-02-04 19:16:58.454669,Russian Caviar,Luxury,2020-02-04
3,1,ccf365,Page View,4,2020-02-04 19:16:58.609142,Lobster,Shellfish,2020-02-04
4,1,ccf365,Add to Cart,5,2020-02-04 19:17:51.72942,Lobster,Shellfish,2020-02-04


In [5]:
event_timeline = events_products.groupby(['date', 'event_name']).count()['user_id'].unstack(1).reset_index()

event_timeline.head()

event_name,date,Ad Click,Ad Impression,Add to Cart,Page View,Purchase
0,2020-01-01,2.0,2.0,22.0,47.0,3.0
1,2020-01-02,6.0,8.0,61.0,146.0,14.0
2,2020-01-03,4.0,4.0,54.0,135.0,12.0
3,2020-01-04,2.0,3.0,42.0,96.0,10.0
4,2020-01-05,2.0,2.0,20.0,47.0,5.0


In [6]:
unique_visits = events_products.groupby('date').nunique()['visit_id']

In [7]:
event_timeline = event_timeline.join(unique_visits, on='date')

In [8]:
event_timeline.rename(columns={"visit_id":"Site Visit"}, inplace=True)

In [9]:
# Event timeline

fig = px.line(event_timeline, x='date', y=event_timeline.columns[1:])

# Add shading to show when marketing campaigns occur
# Half Off - Treat Your Shelf(ish)
fig.add_vrect(x0='2020-02-01', x1='2020-03-31', line_width=0, fillcolor="red", opacity=0.2)

fig.show()

In [10]:
event_summary = events_products.groupby('event_name').count()['user_id'].reset_index().rename(columns={'user_id':'Count'}).sort_values('Count', ascending=False)
event_summary

,event_name,Count
3,Page View,20928
2,Add to Cart,8451
4,Purchase,1777
1,Ad Impression,876
0,Ad Click,702


In [11]:
# Event Summary

fig = px.bar(event_summary, x='event_name', y='Count')
fig.show()

In [12]:
# Number of unique visits
num_unique_visits = sum(unique_visits)
print(num_unique_visits)

num_purchases = len(events_products[events_products['event_name'] == 'Purchase'])

# Percentage of all visits with a purchase event
percentage_purchase = round(num_purchases / num_unique_visits * 100, 1)
print(percentage_purchase)

3574
49.7


In [13]:
# Number of users
num_users = events_products.nunique()['user_id']
print(num_users)

500


In [14]:
def viewed_checkout(columns):
    event_name = columns[0]
    page_name = columns[1]

    if event_name == 'Page View' and page_name == 'Checkout':
        return 1
    else:
        return 0

def made_purchase(column):
    event_name = column

    if event_name == 'Purchase':
        return 1
    else:
        return 0

In [15]:
events_products['viewed_checkout'] = events_products[['event_name', 'page_name']].apply(viewed_checkout, axis=1)

In [16]:
events_products['made_purchase'] = events_products['event_name'].apply(made_purchase)

In [17]:
view_checkout_no_purchase = events_products.groupby('visit_id').sum()[['viewed_checkout', 'made_purchase']]

In [18]:
num_view_checkout_no_purchase = len(view_checkout_no_purchase[(view_checkout_no_purchase['viewed_checkout'] == 1) & (view_checkout_no_purchase['made_purchase'] == 0)])

# Percentage of visits that have a checkout view, but no purchase

percentage_no_purchase = round(num_view_checkout_no_purchase / num_unique_visits * 100, 1)
print(percentage_no_purchase)

9.1


In [19]:
top_pages = events_products[events_products['event_name'] == 'Page View'].groupby('page_name').count().reset_index()[['page_name', 'user_id']].rename(columns={'user_id':'Views'})

top_pages = top_pages.sort_values('Views', ascending=False).iloc[:3]
top_pages

,page_name,Views
1,All Products,3174
3,Checkout,2103
5,Home Page,1782


In [20]:
purchase_visit_id = events_products[events_products['event_name'] == 'Purchase']['visit_id']

purchases = events_products[events_products['visit_id'].isin(purchase_visit_id) & (events_products['event_name'] == 'Add to Cart')]
top_purchases = purchases.groupby('page_name').count().reset_index()[['page_name', 'user_id']].rename(columns={'user_id':'Purchases'}).sort_values('Purchases', ascending=False)
top_purchases = top_purchases[~top_purchases['page_name'].isin(['All Products', 'Checkout', 'Confirmation', 'Home Page'])].iloc[:3]
top_purchases

,page_name,Purchases
4,Lobster,754
5,Oyster,726
2,Crab,719


# Product Funnel Analysis

In [21]:
product_funnel = pd.read_csv('data/funnel_by_product.csv')

In [22]:
# Most viewed

most_viewed = product_funnel.iloc[product_funnel['n_page_views'].argmax()]['page_name']

# Most cart adds

most_adds = product_funnel.iloc[product_funnel['n_added_to_cart'].argmax()]['page_name']

# Most purchases

most_purchases = product_funnel.iloc[product_funnel['purchased_from_cart'].argmax()]['page_name']

# Most Abandons

most_abandons = product_funnel.iloc[product_funnel['abandoned_in_cart'].argmax()]['page_name']

In [23]:
# Purchases / Views Percentage

product_funnel['view_purchase_percent'] = product_funnel['purchased_from_cart'] / product_funnel['n_page_views'] * 100

In [24]:
# Conversion Rates

product_funnel['conversion_view_to_cart'] = product_funnel['n_added_to_cart'] / product_funnel['n_page_views'] * 100

product_funnel['conversion_cart_to_purchase'] = product_funnel['purchased_from_cart'] / product_funnel['n_added_to_cart'] * 100

In [25]:
avg_conversion_view_to_cart = product_funnel['conversion_view_to_cart'].mean()

In [26]:
avg_conversion_cart_to_purchase = product_funnel['conversion_cart_to_purchase'].mean()

In [27]:
product_funnel

,page_id,page_name,product_category,n_page_views,n_added_to_cart,purchased_from_cart,abandoned_in_cart,view_purchase_percent,conversion_view_to_cart,conversion_cart_to_purchase
0,3,Salmon,Fish,1559,938,711,227,45.606158,60.166774,75.799574
1,4,Kingfish,Fish,1559,920,707,213,45.349583,59.012187,76.847826
2,5,Tuna,Fish,1515,931,697,234,46.006601,61.452145,74.865736
3,6,Russian Caviar,Luxury,1563,946,697,249,44.593730,60.524632,73.678647
4,7,Black Truffle,Luxury,1469,924,707,217,48.127978,62.899932,76.515152
5,8,Abalone,Shellfish,1525,932,699,233,45.836066,61.114754,75.000000
6,9,Lobster,Shellfish,1547,968,754,214,48.739496,62.572721,77.892562
7,10,Crab,Shellfish,1564,949,719,230,45.971867,60.677749,75.763962
8,11,Oyster,Shellfish,1568,943,726,217,46.301020,60.140306,76.988335


In [28]:
fig = px.bar(product_funnel, x="page_name", y=product_funnel.columns[3:7])
fig.show()

In [29]:
fig = px.bar(product_funnel, x="page_name", y=product_funnel.columns[7:])
fig.show()

In [30]:
category_funnel = pd.read_csv('data/funnel_by_category.csv')

In [31]:
category_funnel

,product_category,total_page_view,total_added_to_cart,total_purchased,total_abandoned
0,Luxury,3032,1870,1404,466
1,Shellfish,6204,3792,2898,894
2,Fish,4633,2789,2115,674


In [32]:
fig = px.bar(category_funnel, x="product_category", y=category_funnel.columns[1:])
fig.show()

# Campaign Analysis

In [33]:
campaign_analysis = pd.read_csv('data/overall_campaign_metrics.csv')

In [34]:
campaign_analysis.head()

,impression,click,total_visits,total_users,total_views,total_cart_adds,total_purchases,purchase_visit_ratio
0,0,0,2688,500,13443,4034,1040,38.690476
1,1,0,174,148,1115,402,113,64.942529
2,1,1,702,397,6370,4015,624,88.888889


In [35]:
def assign_labels(columns):
    impression = columns[0]
    click = columns[1]

    if impression == 0 and click == 0:
        return 'No Campaign'
    elif impression == 1 and click == 0:
        return 'Ad Impression, No Click'
    elif impression == 1 and click == 1:
        return 'Ad Impression, Ad Click'

campaign_analysis['Campaign Scenario'] = campaign_analysis[['impression', 'click']].apply(assign_labels, axis=1)

In [36]:
campaign_identifier = pd.read_csv('data/campaign_identifier.csv')

In [37]:
campaign_identifier.head()

,campaign_id,products,campaign_name,start_date,end_date
0,1,1-3,BOGOF - Fishing For Compliments,2020-01-01 00:00:00,2020-01-14 00:00:00
1,2,4-5,25% Off - Living The Lux Life,2020-01-15 00:00:00,2020-01-28 00:00:00
2,3,6-8,Half Off - Treat Your Shellf(ish),2020-02-01 00:00:00,2020-03-31 00:00:00


In [38]:
campaign_identifier['end_date'] = pd.to_datetime(campaign_identifier['end_date']).dt.date
campaign_identifier['start_date'] = pd.to_datetime(campaign_identifier['start_date']).dt.date

campaign_identifier['duration'] = campaign_identifier['end_date'] - campaign_identifier['start_date']
campaign_identifier['duration'] = campaign_identifier['duration'].dt.days

campaign_identifier['products_campaigned'] = pd.Series(['Salmon, Kingfish, Tuna', 'Russian Caviar, Black Truffle', 'Abalone, Lobster, Crab'])

In [39]:
events_products = pd.read_csv('data/combined_events_products.csv')
events_products['date'] = pd.to_datetime(events_products['event_time']).dt.date

In [40]:
total_num_days = events_products['date'].nunique()
total_num_days

131

In [41]:
days_campaign = campaign_identifier['duration'].sum()
days_campaign

85

In [42]:
campaign_analysis['days'] = pd.Series([(total_num_days - days_campaign), days_campaign, days_campaign])

campaign_analysis['total_visits_per_day'] = campaign_analysis['total_visits'] / campaign_analysis['days']
campaign_analysis['total_users_per_day'] = campaign_analysis['total_users'] / campaign_analysis['days']
campaign_analysis['total_views_per_day'] = campaign_analysis['total_views'] / campaign_analysis['days']
campaign_analysis['total_cart_adds_per_day'] = campaign_analysis['total_cart_adds'] / campaign_analysis['days']
campaign_analysis['total_purchases_per_day'] = campaign_analysis['total_purchases'] / campaign_analysis['days']

In [43]:
campaign_analysis.head()

,impression,click,total_visits,total_users,total_views,total_cart_adds,total_purchases,purchase_visit_ratio,Campaign Scenario,days,total_visits_per_day,total_users_per_day,total_views_per_day,total_cart_adds_per_day,total_purchases_per_day
0,0,0,2688,500,13443,4034,1040,38.690476,No Campaign,46,58.434783,10.869565,292.239130,87.695652,22.608696
1,1,0,174,148,1115,402,113,64.942529,"Ad Impression, No Click",85,2.047059,1.741176,13.117647,4.729412,1.329412
2,1,1,702,397,6370,4015,624,88.888889,"Ad Impression, Ad Click",85,8.258824,4.670588,74.941176,47.235294,7.341176


In [44]:
fig = px.bar(campaign_analysis, x="Campaign Scenario", y=campaign_analysis.columns[2:7])
fig.show()

In [45]:
fig = px.bar(campaign_analysis, x="Campaign Scenario", y=campaign_analysis.columns[10:])
fig.show()

In [46]:
# purchase_visit_ratio No campaign

campaign_analysis[['Campaign Scenario', 'purchase_visit_ratio']].iloc[0][1]

38.69047619047619

In [47]:
campaign_comparison = pd.read_csv('data/campaign_comparison.csv')

In [48]:
campaign_comparison.head()

,campaign_name,total_visits,total_users,total_views,total_cart_adds,total_purchases,purchase_visit_ratio,cart_conversion_ratio,purchase_conversion_ratio
0,NaN,512,183,3061,1243,268,52.343750,40.607645,21.560740
1,25% Off - Living The Lux Life,404,160,2434,991,202,50.000000,40.714873,20.383451
2,Half Off - Treat Your Shellf(ish),2388,449,13897,5592,1180,49.413735,40.238900,21.101574
3,BOGOF - Fishing For Compliments,260,103,1536,625,127,48.846154,40.690104,20.320000


In [49]:
campaign_comparison['campaign_name'].fillna('No Campaign', inplace=True)

In [50]:
campaign_comparison['days'] = pd.Series(
    [(total_num_days - days_campaign),
    campaign_identifier[campaign_identifier['campaign_name'] == '25% Off - Living The Lux Life']['duration'].values[0],
    campaign_identifier[campaign_identifier['campaign_name'] == 'Half Off - Treat Your Shellf(ish)']['duration'].values[0],
    campaign_identifier[campaign_identifier['campaign_name'] == 'BOGOF - Fishing For Compliments']['duration'].values[0]]

)

campaign_comparison['total_visits_per_day'] = campaign_comparison['total_visits'] / campaign_comparison['days']
campaign_comparison['total_users_per_day'] = campaign_comparison['total_users'] / campaign_comparison['days']
campaign_comparison['total_views_per_day'] = campaign_comparison['total_views'] / campaign_comparison['days']
campaign_comparison['total_cart_adds_per_day'] = campaign_comparison['total_cart_adds'] / campaign_comparison['days']
campaign_comparison['total_purchases_per_day'] = campaign_comparison['total_purchases'] / campaign_comparison['days']

In [51]:
fig = px.bar(campaign_comparison, x='campaign_name', y=campaign_comparison.columns[1:6])
fig.show()

In [52]:
fig = px.bar(campaign_comparison, x='campaign_name', y=campaign_comparison.columns[10:])
fig.show()

In [55]:
fig = px.bar(campaign_comparison, x='campaign_name', y=campaign_comparison.columns[6:9])
fig.show()